In [5]:
%load_ext autoreload
%autoreload 2
import os
import sys
import json
import numpy as np
import torch
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import pyepo
import time
import argparse
from torch_models import PyEPONegativeBinomialRegressionModel, deterministic_bpr
from data_loader import load_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
best_cook_spo = '/cluster/tufts/hugheslab/kheuto01/pyepo_exps/20250330_run/cook_spo+_step1e-01'
data_dir = '/cluster/tufts/hugheslab/kheuto01/code/decision-aware-topk/data/aerial_surv/'
seed = 360
method_name = 'spo+'
pg_sigma = 1
batch_size = 4
K=50
device='cpu'
step_size = 1e-3


In [8]:
# Set random seed for reproducibility
if seed is not None:
    torch.manual_seed(seed)
    np.random.seed(seed)

X_train, y_train, train_time = load_data(data_dir, 'train', device='cpu')
# add train_time [B,S] to the end of X_train [B,S,F]
X_train = torch.cat([X_train, train_time.unsqueeze(-1)], dim=-1)
capacities = [K]
num_locations = X_train.shape[1]
weights = np.ones((1, num_locations))
optmodel = pyepo.model.grb.knapsackModel(weights, capacities)
# get training data set
dataset_train = pyepo.data.dataset.optDataset(optmodel, X_train, y_train)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

X_val, y_val, val_time = load_data(data_dir, 'valid', device='cpu')
X_val = torch.cat([X_val, val_time.unsqueeze(-1)], dim=-1)
dataset_val = pyepo.data.dataset.optDataset(optmodel, X_val, y_val)
loader_val = DataLoader(dataset_val, batch_size=len(dataset_val), shuffle=False)

# Initialize model
model = PyEPONegativeBinomialRegressionModel(
    num_locations=X_train.shape[1],
    num_fixed_effects=X_train.shape[2]-1, device=device
).to(device)

if method_name =='spo+':
    spop = pyepo.func.SPOPlus(optmodel, processes=1)
    loss_func = spop
elif method_name == 'pg':
    loss_func = pyepo.func.perturbationGradient(optmodel, sigma=pg_sigma, two_sides=False, processes=1)
elif method_name == 'ltr':
    loss_func = pyepo.func.LTR(optmodel, processes=1)
else:
    raise ValueError(f"Unknown method {method_name}")

# Setup optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=step_size)

    # Initialize metric tracking with separate epoch tracking for validation
metrics = {
    'train': {
        'epochs': [], 
        'loss': [], 
        'nll': [], 
        'deterministic_bpr': [],
        'regret': [],
    },
    'val': {
        'epochs': [], 
        'loss': [], 
        'nll': [], 
        'deterministic_bpr': [],
        'regret': [],
    },
    'times': []
}

best_val_loss = float('inf')

Restricted license - for non-production use only - expires 2026-11-23
Optimizing for optDataset...


100%|██████████| 15/15 [00:00<00:00, 270.38it/s]


Optimizing for optDataset...


100%|██████████| 6/6 [00:00<00:00, 627.89it/s]


Num of cores: 1


In [ ]:
# update model parameters, set beta and beta_0 to zeroes, b_0 to zeroes, and b_1 to ones
model.beta.data.zero_()
model.beta_0.data[:-5].zero_()
model.beta_0.data[-5:].fill_(1.0)
model.b_0.data.zero_()
model.b_1.data.zero_()


tensor([1., 1., 1.,  ..., 1., 1., 1.])